# Tests

In [1]:
import pyhelayers
import utils 
import copy
utils.verify_memory()
# Print the ciphertext content for demo purposes
pyhelayers.get_print_options().print_encrypted_content=True 
print("Imported pyhelayers", pyhelayers.VERSION)

Imported pyhelayers 1.3.0


In [2]:
num_slots = 8192 # Number of slots per ciphertext
multiplication_depth = 2 # Allow 2 levels of multiplications
fractional_part_precision = 40 # Set the precision to 1/2^40.
integer_part_precision = 20 # Set the largest number to 2^20.
security_level = 128

requirement = pyhelayers.HeConfigRequirement(
    num_slots,
    multiplication_depth,
    fractional_part_precision,
    integer_part_precision,
    security_level)

he_context = pyhelayers.DefaultContext()
he_context.init(requirement)

In [3]:
# Create the Encoder using the context.
encoder = pyhelayers.Encoder(he_context)

# Encode a vector.
v1 = [8.0, 2.0, 3.0]
p1 = encoder.encode(v1)
c1 = encoder.encode_encrypt(v1)
v2 = [2,3,4]
p2 = encoder.encode(v2)
c2 = encoder.encode_encrypt(v2)
cAdd = copy.deepcopy(c1)
cAdd.add(c2)
cSub = copy.deepcopy(c1)
cSub.sub(c2)
cMult = copy.deepcopy(c1)  # relinearizes automatically
cMult.multiply(c2)
cSq = copy.deepcopy(c1)
cSq.square()
cNeg = copy.deepcopy(c1)
cNeg.negate()
# NOTE: HELAYERS DOES NOT HAVE A DESIGNATED POWER FCT
cPow = copy.deepcopy(c1)
for i in range(3-1):
    cPow.multiply(c1)
cRotR = copy.deepcopy(c1)
cRotR.rotate(c1.slot_count() - 1)
cRotL = copy.deepcopy(c1)
cRotL.rotate(1)
# Ciphertext-plaintext ops
cpSum = copy.deepcopy(c1)
cpSum.add_plain(p2)
cpSub = copy.deepcopy(c1)
cpSub.sub_plain(p2)
cpMult = copy.deepcopy(c1)
cpMult.multiply_plain(p2)

# Decode the encoded vector.
d1 = encoder.decrypt_decode_double(c1)
d2 = encoder.decrypt_decode_double(c2)

dAdd = encoder.decrypt_decode_double(cAdd)
dSub = encoder.decrypt_decode_double(cSub)
dMult = encoder.decrypt_decode_double(cMult)
dSq = encoder.decrypt_decode_double(cSq)
dNeg = encoder.decrypt_decode_double(cNeg)
dPow = encoder.decrypt_decode_double(cPow)
dRotR = encoder.decrypt_decode_double(cRotR)
dRotL = encoder.decrypt_decode_double(cRotL)
dcpSum = encoder.decrypt_decode_double(cpSum)
dcpSub = encoder.decrypt_decode_double(cpSub)
dcpMult = encoder.decrypt_decode_double(cpMult)

In [4]:
print('Initial vector:   {}'.format(v1))
print('Encrypted vector: {}'.format(c1))
print('Decoded vector:   {}'.format(d1[:3]))
print('d2:    {}'.format(d2[:3]))
print()
print('dAdd:  {}'.format(dAdd[:3]))
print('dSub:  {}'.format(dSub[:3]))
print('dMult: {}'.format(dMult[:3]))
print('dSq: {}'.format(dSq[:3]))
print('dNeg: {}'.format(dNeg[:3]))
print('dPow: {}'.format(dPow[:3]))
print('dRotR: {}'.format(dRotR[:3]))
print('dRotL: {}'.format(dRotL[:3]))
print('dcpSum: {}'.format(dcpSum[:3]))
print('dcpSub: {}'.format(dcpSub[:3]))
print('dcpMult: {}'.format(dcpMult[:3]))

Initial vector:   [8.0, 2.0, 3.0]
Encrypted vector: CIPHERTEXT [703587965733386627 1080681253202402890 303379446208550837 806821731153072107 ...  690804556267 ]
Decoded vector:   [8.         1.99999999 3.        ]
d2:    [2. 3. 4.]

dAdd:  [10.          4.99999999  7.        ]
dSub:  [ 6. -1. -1.]
dMult: [16.          5.99999998 12.        ]
dSq: [63.99999998  3.99999998  9.00000001]
dNeg: [-8.         -1.99999999 -3.        ]
dPow: [511.99999959   7.99999994  27.00000003]
dRotR: [1.24348731e-06 7.99999992e+00 2.00000034e+00]
dRotL: [1.99999947e+00 3.00000013e+00 2.11319314e-09]
dcpSum: [10.          4.99999999  7.        ]
dcpSub: [ 6.         -1.00000001 -1.        ]
dcpMult: [16.          5.99999999 12.        ]


In [5]:
import time
import random

start = time.time()

num_numbers = 1
v1 = [0] * num_numbers
for i in range(num_numbers):
    v1[i] = random.random() * 2
p1 = encoder.encode(v1)
c1 = encoder.encode_encrypt(v1)
print(v1)

v2 = [0] * num_numbers
for i in range(num_numbers):
    v2[i] = random.random() * 2
p2 = encoder.encode(v2)
c2 = encoder.encode_encrypt(v2)
print(v2)

num_mults = 2
for i in range(num_mults):
    c1.multiply(c2)
    
rMul = encoder.decrypt_decode_double(c1)
print('rMul:  {}'.format(rMul[:3]))

end = time.time()
print(end - start)

[0.03986796221091615]
[1.5534933759168017]
rMul:  [8.94367903e-09 9.63465970e-11 3.92240576e-10]
0.04662680625915527
